# 0. Carregando os dados e bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.set_option('display.float_format', lambda x: '%.2f' % x)
from matplotlib import pyplot as plt
#Foram adicionados por conta de erro de renderização no mapa (ex1.10), utilizando o VS Code. Caso sua IDE não tenha o erro, não é necessário
#Erro: No renderer could be found for mimetype "application/vnd.plotly.v1+json"
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [ ]:
data = pd.read_csv('datasets/kc_house_data.csv')

# 1. Respondendo às perguntas de negócio

## 1.1 Qual o número de imóveis por ano de construção?

In [ ]:
data.groupby('yr_built')[['id']].count()

#### Outra forma de fazer

In [ ]:
data[['id', 'yr_built']].groupby('yr_built').count()

#essa sintaxe pode ser usada para os exercícios da seção 1

## 1.2 Qual o menor número de quartos por ano de construção dos imóveis?


In [ ]:
data.groupby('yr_built')[['bedrooms']].min()

## 1.3 Qual o preço de compra mais alto por cada número de quartos?

In [ ]:
data.groupby('bedrooms')[['price']].max()

## 1.4 Qual a soma de todos os preços de compra por cada número de quartos?

In [ ]:
data.groupby('bedrooms')[['price']].sum()

## 1.5 Qual a soma de todos os preços de compra pelo número de quartos e banheiros?

In [ ]:
data.groupby(['bedrooms', 'bathrooms'])[['price']].sum()

## 1.6 Qual o tamanho médio das salas dos imóveis por ano de construção?

In [ ]:
data.groupby('yr_built')[['sqft_living']].mean()

## 1.7 . Qual o tamanho mediano das salas dos imóveis por ano de construção?

In [ ]:
data.groupby('yr_built')[['sqft_living']].median()

## 1.8 Qual o desvio-padrão do tamanho das salas dos imóveis por ano de construção?

In [ ]:
data.groupby('yr_built')[['sqft_living']].std()

## 1.9 Como é o crescimento médio preços de compra dos imóveis, por dia e semana do ano?

- Primeiro gráfico (crescimento x ano)

In [ ]:
#Eixo x: data (ano)
#Eixo y: preço médio
#Tipo de gráfico: barras

from matplotlib import pyplot as plt

data['year'] = pd.to_datetime( data['date'] ).dt.year

by_year = data.groupby('year')[['price']].sum().reset_index()
#data[['price', 'year']].groupby('year').sum().reset_index() -> também dá certo

#Plotar o gráfico
plt.bar(by_year ['year'], by_year['price'])

- Segundo gráfico (crescimento x dia)

In [ ]:
#Eixo x: data (dia)
#Eixo y: preço médio
#Tipo de gráfico: barras

from matplotlib import pyplot as plt

data['day'] = pd.to_datetime( data['date'] )

by_day= data.groupby('day')[['price']].mean().reset_index()
#data[['price', 'year']].groupby('year').sum().reset_index() -> também dá certo

#Plotar o gráfico
plt.figure(figsize=(10,6))
plt.plot(by_day ['day'], by_day['price'])

- Terceiro gráfico (crescimento x semana)

In [ ]:
#Eixo x: data (ano)
#Eixo y: preço médio
#Tipo de gráfico: barras

from matplotlib import pyplot as plt

data['year_week'] = pd.to_datetime( data['date'] ).dt.strftime('%Y-%U') #%u -> semana começa no domingo

by_year_week= data.groupby('year_week')[['price']].mean().reset_index()

#Plotar o gráfico
plt.figure(figsize=(10,5))
plt.plot(by_year_week ['year_week'], by_year_week['price'])
plt.xticks(rotation=60); #Importante o ; para não mostrar o texto da rotação

- Dash com os 3 gráficos

In [ ]:
#Desenho do dash
from matplotlib import gridspec

fig = plt.figure(figsize=(20,12))
specs = gridspec.GridSpec(ncols=2, nrows=2, figure=fig)

ax1 = fig.add_subplot(specs[0,:]) #Primeira linha
ax2 = fig.add_subplot(specs[1,0]) #Primeira coluna, segunda linha
ax3 = fig.add_subplot(specs[1,1]) #Segunda coluna, segunda linha


#Primeiro gráfico
data['year'] = pd.to_datetime( data['date'] ).dt.year
by_year = data.groupby('year')[['price']].sum().reset_index()
ax1.bar(by_year ['year'], by_year['price'])

#Segundo gráfico
data['day'] = pd.to_datetime( data['date'] )
by_day= data.groupby('day')[['price']].mean().reset_index()
ax2.plot(by_day ['day'], by_day['price'])

#Terceiro gráfico
data['year_week'] = pd.to_datetime( data['date'] ).dt.strftime('%Y-%U')
by_year_week= data.groupby('year_week')[['price']].mean().reset_index()
ax3.plot(by_year_week ['year_week'], by_year_week['price'])
plt.xticks(rotation=60);

## 1.10 Eu gostaria de olhar no mapa e conseguir identificar as casas com o maior preço.

In [ ]:
houses = data[['id', 'lat','long', 'price']]

fig = px.scatter_mapbox(houses,
                        lat = 'lat',
                        lon = 'long',
                        size = 'price',
                        color_continuous_scale = px.colors.cyclical.IceFire,
                        size_max=15,
                        zoom=10
)
fig.update_layout(mapbox_style = 'open-street-map')
fig.update_layout(height=600, margin={'r':0, 't':0, 'l':0, 'b':0})
fig.show()

# 2. Exercícios

## 2.1 Crie uma nova coluna chamada: “dormitory_type”
- Se o valor da coluna “bedrooms” for igual à 1 => ‘studio’
- Se o valor da coluna “bedrooms” for igual a 2 => ‘apartament’
- Se o valor da coluna “bedrooms” for maior que 2 => ‘house’

In [ ]:
data['dormitory_type'] = data['bedrooms'].apply ( lambda x: 'studio' if x== 1 else
                                                            'apartment' if x == 2 else
                                                            'house' if x > 2 else
                                                            'NDA')

## 2.2 Faça um gráfico de barras que represente a soma dos preços pelo número de quartos.

In [ ]:
#Eixo x: número de quartos
#Eixo y: soma dos preços
#Tipo de gráfico: barras
by_bedrooms = data.groupby('bedrooms')[['price']].sum().reset_index()

#Plotar o gráfico
#plt.bar(x, y)
plt.bar( by_bedrooms['bedrooms'], by_bedrooms['price'], color ='blue')

## 2.3 Faça um gráfico de linhas que represente a média dos preços pelo ano construção dos imóveis.

In [ ]:
#Eixo x: data (ano)
#Eixo y: preço médio
#Tipo de gráfico: barras

#Converter a coluna de data para formato de data e extrair o ano
data['year'] = pd.to_datetime( data['date'] ).dt.year

by_year = data.groupby('year')['price'].mean().reset_index()

#Plotar gráfico
#plt.plot( x, y )
plt.plot( by_year['year'], by_year['price'] )



## 2.4 Faça um gráfico de barras que represente a média dos preços pelo tipo dos dormitórios.

In [ ]:
#Eixo x: tipo de dromitório
#Eixo y: preço médio
#Tipo de gráfico: barras

by_dorm_type = data.groupby('dormitory_type')['price'].mean().reset_index()

plt.bar( by_dorm_type['dormitory_type'], by_dorm_type['price'] )


## 2.5 . Faça um gráfico de linha que mostre a evolução da média dos preços pelo ano da reforma dos imóveis, a partir do ano de 1930.


## 2.6  Faça um tabela que mostre a média dos preços por ano de construção e tipo de dormitórios dos imóveis.

## 2.7 Crie um Dashboard com os gráficos das questões 02, 03, 04 ( Dashboard: 1 Linha e 2 colunas)

## 2.8 Crie um Dashboard com os gráficos das perguntas 02, 04 ( Dashboard: 2 colunas )

## 2.9 Crie um Dashboard com os gráficos das perguntas 03, 05 ( Dashboard: 2 Linhas )


## 2.10 Faça um gráfico com o tamanho dos pontos sendo igual ao tamanho da sala de estar